In [8]:
import base64
import datetime
import os
import markdown2
import pandas as pd
from langgraph.graph import END, StateGraph
from LLM_get_folder import get_local_folder
from utilities.wordpress_tools import (
    check_insert_image,
    get_rewrite_post_ID,
    insert_keyword_url,
    post_wordpress_post,
    retrieve_wordpress_post,
    tags_to_IDs,
    update_summary_qa,
    replace_amcharts_code,
    replace_videos_code,
    html_to_markdown,
)
from schemas.schemas_rewrite import GraphState, ReviseOutput, PostTitleTags
from utilities.llm_wrapper import llm_wrapper, llm_wrapper_raw, tokens_count
from utilities.web_loader_wrapper import web_loader_docs
from utilities.web_search_wrapper import web_search_wrapper

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "forwardpathway_post_rewrite"
wp_url = "https://www.forwardpathway.com/wp-json/wp/v2"
wp_post_url = wp_url + "/posts"
wp_media_url = wp_url + "/media"
wp_tag_url = wp_url + "/tags"

user_id = os.environ["wordpress_username"]
# user app password can be created in the user/edit user/application password
user_app_password = os.environ["wordpress_pass"]

credentials = user_id + ":" + user_app_password
token = base64.b64encode(credentials.encode())
header = {"Authorization": "Basic " + token.decode("utf-8")}

In [9]:
def get_post_content(state):
    post_ID = state["post_ID"]
    post_url = "https://www.forwardpathway.com/" + str(post_ID)
    response = retrieve_wordpress_post(post_ID)
    post_title = response.json()["title"]["rendered"]
    raw_html = response.json()["content"]["rendered"]
    feature_image_ID = response.json()["featured_media"]
    tags = response.json()["tags"]
    original_content = html_to_markdown(raw_html)
    return {
        "post_title": post_title,
        "raw_html": raw_html,
        "original_content": original_content,
        "URL": post_url,
        "tags": tags,
        "feature_image_ID": feature_image_ID,
    }

In [10]:
def get_revise_comments(state):
    post_title = state["post_title"]
    post_content = state["original_content"]
    today = datetime.datetime.now().strftime("%Y年%m月%d日")
    system_prompt = f"""今天的日期是{today}，下面将给出一篇网站文章内容和该文章标题，请对该文章的内容且只对内容提出具体的修改意见和搜索词条。\
        修改意见可以包含但不限于以下几个方面：使用数据过时、前后逻辑不清晰、用词是否恰当、前后文是否对应。
        修改意见要求：
        1. 内容扩展建议：如果文章长度少于3000个中文字，请提供增加文章长度的建议，并详细描述应增加的内容及搜索该内容的具体搜索词条。
        2. 数据和信息的准确性：检查文章中的数据和信息是否准确和最新。如果发现数据或信息过时，请提出具体的修改建议及搜索更新信息的搜索词条，\
            该搜索词条需要包含该信息的具体内容以便搜索时间。
        3. 信息具体时间缺失：检查文章中事件或信息的具体时间，如果有缺失或不准确的，请给出具体的修改建议及搜索该事件或信息具体时间的搜索词条。\
            比如时间缺少年份的请给出补充时间年份的建议，同时搜索词条应该给出能搜索到该事件具体时间的搜索词条。\
                注意：事件的时间不要默认为今年或今天，应该给出具体搜索词条去搜索该事件的时间。
        4. 逻辑和结构：评估文章的逻辑和结构，确保前后文连贯，文章内容契合文章标题和主题。如果发现逻辑不清晰或结构混乱的地方，请指出并给出修改建议及搜索相关信息的搜索词条。
        5. 用词和语言：检查文章的用词是否恰当，语言是否流畅。如果发现用词不当或语言不流畅的地方，请提出修改建议及搜索相关信息的搜索词条。
        6. 引用和出处：检查文章中的引用和出处是否清晰明确。如果发现引用不清楚或缺乏出处的地方，请提出修改建议及搜索相关信息的搜索词条。
        7. 修改意见要具体：请具体指出文章中哪些部分需要修改，详细描述不好的地方，如数据过时、前后逻辑不清晰、用词不当等并对应的给出搜索相关信息的搜索词条。
        搜索词条要求：修改意见与搜索词条需一一对应。每一条修改意见应对应一个具体的搜索词条。搜索词条必须是英文，且应能在Google上搜索到相应的资料。
        输出格式：请给出3条修改意见，并将每一条修改意见与搜索词条放入对应的输出数组中。确保每条修改意见具体明确，并对应相关的搜索词条。"""
    user_prompt = f"文章标题:{post_title}\n\n文章内容: {post_content}"
    response = llm_wrapper(system_prompt, user_prompt, ReviseOutput)
    return {"revises": response.revises}

In [11]:
def revise_post(state):
    concat_max = 3
    MAX_DOC_PER_SEARCH = 4
    MAX_TOKEN_LENGTH = 125000
    post_title = state["post_title"]
    revises = state["revises"]
    revise_content = state["original_content"]
    revise_n = 0
    today = datetime.datetime.now().strftime("%Y年%m月%d日")
    system_common = f"""任务说明：你是一名专业的网站内容编辑。下面将给出一篇网站文章内容和该文章标题，并提供一条修改意见以及与该修改意见相关的一些参考资料。\
    请根据修改意见和参考资料对原文章内容进行修改，没有修改意见的部分请保持原文内容。最终输出结果必须是markdown格式输出，但是不要在文章最前端添加markdown标志。今天的日期是{today}
    修改要求：
    1. 内容更新：请参考修改意见和相关资料对原文进行修改。如果更新内容涉及表格数据，请对整个表格进行更新，且更新内容必须来自参考资料。\
        如果参考资料中不包含关键信息，请删除原表格中的相关内容。修改后的文章内容要尽量贴合文章标题和原文章主题。\
            在文章的最开始位置添加这次更新的更新时间和更新内容简短概述，更新记录最多保留不同日期的3条记录，同一天的修改记录请合并处理，这段更新记录必须在文章的最开始位置。
    2. 信息具体时间更新：如果更新内容涉及具体时间的，修改年份和时间必须从参考资料中获取且必须要准确无误，如果参考资料中不包含所需要的具体时间则不更新该内容时间。
    3. 排版要求：修改后的文章需输出修改后的全文内容，且仅输出文章内容，不要包含文章标题或者其他内容。对遇到的表格进行表格排版，各级标题统一格式排版。
    4. 中文名翻译：如果原文中提到的美国大学只有英文名，请添加中文名翻译。
    5. 图片链接处理：如果原文中有图片链接请保留原文中的图片链接，不要删除该图片链接，也不要改动图片链接，请保留原文中的所有图片链接。
    6. 数据图代码处理：如果原文中有类似[amcharts id="chart-xx"]的代码，"这些代码请保留原有代码，不可以有任何改动"。"""
    system_diff = [
        """\n7. 参考标记和编号：文章原文末尾如果有参考资料部分的，请先删除文章末尾原有的参考资料，同时删除原文中所有参考资料编号，\
            参考资料编号也可能是"【1】"，"【2】"，"【3】"等。然后如果修改是参考了搜索资料，请在文章内容中添加参考标记与编号，并在文章末尾添加参考过的资料与编号，\
                文章内容中的参考资料编号需要与文章末尾的编号一一对应。如果没做修改或者修改没有参考的搜索资料不要添加。参考资料部分需统一排版并编号，\
                    参考资料如果有重复请删除重复的资料，如果文章末尾参考资料编号做了改动，与之对应的文章内容中的参考资料标号也要做修改。""",
        """\n7. 参考标记和编号：如果修改是参考了搜索资料，请在文章内容中添加参考标记与编号，并在文章末尾添加参考过的资料与编号，\
            文章内容中的参考资料编号需要与文章末尾的编号一一对应。如果没做修改或者修改没有参考的搜索资料不要添加。参考资料部分需统一排版并编号，\
                参考资料如果有重复请删除重复的资料，如果文章末尾参考资料编号做了改动，与之对应的文章内容中的参考资料标号也要做修改。""",
    ]
    for revise in revises:
        original_content = revise_content
        comment = revise.comment
        search_query = revise.search_query
        results = web_search_wrapper(
            query=search_query + " -filetype:pdf",
            type="search",
        )
        docs = []
        print("--------get search results done!---------------")
        system_prompt = system_common + system_diff[revise_n >= 1]
        user_prompt = f"文章标题:{post_title}\n\n原文章内容: {revise_content}\n\n修改意见:{comment}\n\n参考资料:{docs}"
        prompt_tokens = tokens_count(system_prompt + user_prompt)
        search_result_count = 0
        total_token = 0
        for x in results["organic"]:
            if search_result_count >= MAX_DOC_PER_SEARCH:
                break
            try:
                new_doc = web_loader_docs(x["link"])
                token_length = tokens_count(str(new_doc[0]))
                if token_length <= 40000:
                    if (total_token + token_length + prompt_tokens) >= MAX_TOKEN_LENGTH:
                        break
                    docs = docs + new_doc
                    search_result_count += 1
                    total_token = total_token + token_length
            except:
                continue
        print("--------load docs done!---------------")

        response = llm_wrapper_raw(
            system_prompt,
            user_prompt,
            timeout=120,
            temperature=0.2,
        )
        revise_content = response.choices[0].message.content
        stop_reason = response.choices[0].finish_reason
        print("----stop reason:", stop_reason, "----")
        concat_count = 0
        while stop_reason == "length":
            print("---continue generate next part---")
            if concat_count >= concat_max:
                raise Exception("max count reached")
            system_prompt = (
                "你已经生成了第一部分输出内容，请根据下面相同的任务需求及第一部分输出内容继续生成剩余的内容。\n\n"
                + system_common
                + system_diff[revise_n >= 1]
            )
            user_prompt = f"文章标题:{post_title}\n\n原文章内容: {original_content}\n\n修改意见:{comment}\n\n参考资料:{docs}\n\n第一部分输出内容：{revise_content}"
            response = llm_wrapper_raw(
                system_prompt,
                user_prompt,
                timeout=120,
                temperature=0.2,
            )
            revise_content = revise_content + response.choices[0].message.content
            stop_reason = response.choices[0].finish_reason
            concat_count += 1
        revise_n += 1
        print(f"-----------{revise_n}x revise done!---------------")
    return {"revised_content": revise_content}

In [12]:
def update_post(state):
    post_ID = state["post_ID"]
    post_title = state["post_title"]
    raw_content = state["revised_content"]
    tags = state["tags"]
    feature_image_ID = state["feature_image_ID"]
    system_prompt = """你的角色是一名网站编辑，用户给你内一篇关于美国留学的文章内容与这篇文章的老标题，请根据文章内容更新老标题，标题风格与内容需要与老标题相似，\
        只根据提供的文章内容对老标题做出必要的修改。修改文章标题中的时间需要谨慎，忽略文章最开始的更新时间和更新记录，尽量根据文章主体内容来修改标题。\
            最后输出内容只包含更新后的标题，不要包含其他内容，比如不要包含'更新后的标题'这类内容，也不要包含老标题，只输出更新后的新标题。\
                同时根据文章内容及现有标题生成一些与文章内容相关的标签。"""
    user_prompt = f"文章老标题:{post_title}\n\n文章内容: {raw_content}"
    try:
        response = llm_wrapper(system_prompt, user_prompt, PostTitleTags)
        new_title = response.title
        generated_tags = tags_to_IDs(response.tags)
    except:
        new_title = post_title
        generated_tags = set()
    content = markdown2.markdown(raw_content, extras=["tables", "footnotes"])
    content = replace_amcharts_code(content)
    content = replace_videos_code(content)
    content, link_tags = insert_keyword_url(content)
    content = check_insert_image(content, feature_image_ID)
    tags = set(tags) | generated_tags | link_tags
    post_wordpress_post(
        post_ID=post_ID, post_title=new_title, post_body=content, tags=tags
    )
    update_summary_qa(post_ID, raw_content)
    folder = get_local_folder()
    filepath = os.path.join(folder, "post_rewrite.csv")
    df = pd.DataFrame(
        [[post_ID, post_title, new_title, state["raw_html"], content]],
        columns=["post_ID", "post_title", "new_title", "raw_html", "revised"],
    )
    df.to_csv(filepath, mode="a", index=False, header=False)
    return

In [13]:
######################## Build LangGraph ####################################
workflow = StateGraph(GraphState)
workflow.add_node("get_post_content", get_post_content)
workflow.add_node("get_revise_comments", get_revise_comments)
workflow.add_node("revise_post", revise_post)
workflow.add_node("update_post", update_post)

workflow.set_entry_point("get_post_content")
workflow.add_edge("get_post_content", "get_revise_comments")
workflow.add_edge("get_revise_comments", "revise_post")
workflow.add_edge("revise_post", "update_post")
workflow.add_edge("update_post", END)

app = workflow.compile()

from IPython.display import Image as IPImage
from IPython.display import display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

display(
    IPImage(
        app.get_graph().draw_mermaid_png(
            curve_style=CurveStyle.BASIS,
            node_colors=NodeStyles(
                first="fill:#FDFFB6",
                last="fill:#FFADAD",
                default="fill:#CAFFBF,line-height:1",
            ),
            draw_method=MermaidDrawMethod.API,
        ),
        width=300,
    )
)

img = app.get_graph().draw_mermaid_png(
    curve_style=CurveStyle.BASIS,
    node_colors=NodeStyles(
        first="fill:#FDFFB6",
        last="fill:#FFADAD",
        default="fill:#CAFFBF,line-height:1",
    ),
    draw_method=MermaidDrawMethod.API,
)
with open("post_rewrite_flow.png", "wb") as png:
    png.write(img)

In [14]:
app.invoke({"post_ID": get_rewrite_post_ID()})
# app.invoke({"post_ID": 135815})

--------get search results done!---------------
--------load docs done!---------------
----stop reason: stop ----
-----------1x revise done!---------------
--------get search results done!---------------
--------load docs done!---------------
----stop reason: stop ----
-----------2x revise done!---------------
--------get search results done!---------------
--------load docs done!---------------
----stop reason: stop ----
-----------3x revise done!---------------


{'post_ID': 107060,
 'URL': 'https://www.forwardpathway.com/107060',
 'post_title': '普渡大学草地合作伙伴关系项目：提升农场生产力与生态系统服务',
 'raw_html': '<p><a href="https://www.forwardpathway.com/10360">普渡大学西拉法叶分校</a>（2024USNews<a href="https://www.forwardpathway.com/ranking">美国大学排名</a>：43）新闻报道，草地合作伙伴关系邀请南印第安纳州的牧草畜牧生产者参与一个农场示范/研究项目，以帮助实施草地管理实践。该项目旨在提高以冷季草为主的农场生产力，特别是高羊茅草。参与者需位于“高羊茅带”区域内。普渡大学西拉法叶分校推广部将审查报名表并联系符合条件的生产者，推广部将与生产者合作实施多项草地管理实践，如建立本地暖季草牧场、改进放牧管理、间播豆科植物、建立多年生田地缓冲区、建立林牧结合系统、用生物炭或石膏改良牧场土壤等。这些实践将根据每个生产者的农场和管理目标进行定制，并由推广部在四年内评估实施效果。项目预算将补贴实施成本。该研究项目由美国农业部自然资源保护局和气候智能商品合作伙伴关系资助，除印第安纳州外，还有阿拉巴马州、阿肯色州、肯塔基州、密苏里州、北卡罗来纳州、南卡罗来纳州、田纳西州和弗吉尼亚州参与。普渡大学西拉法叶分校农学教授兼推广牧草专家基思·约翰逊表示，参与该项目是成为“高羊茅带”地区动态和好奇团队的一部分的绝佳机会。</p>\n<p>普渡大学西拉法叶分校推广部在该项目中扮演了重要角色。普渡大学西拉法叶分校扩展社区发展部正在通过大湖环境正义繁荣社区技术援助中心（TCTAC）在印第安纳州领导一系列研讨会。这些研讨会旨在帮助组织和地方政府导航联邦资金机会，以获取用于提高社区生活质量、清洁能源、污染清理和绿色劳动力发展等项目的资源。参与者将学习如何与州、地区和国家技术援助伙伴及相关资源联系，以获取联邦资金，并参与撰写和管理资助提案的技能培训。研讨会将在多个县举行，包括马里恩县、维戈县、艾伦县、哈里森县、斯宾塞县、巴塞洛缪县、韦恩县和波特县。普渡大学西拉法叶分校扩展社区发展部的助理项目负责人卡拉·萨拉查表示，这些研讨会支持旨在建立地方能力